In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([0])


In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2871, -0.2294, -0.2782,  0.0446, -0.3867, -0.3955,  0.4203,  0.7009,
          0.3356,  0.3799, -0.0280, -0.3232, -0.6769,  0.0238,  0.2320, -0.0439,
          0.3813, -0.0608, -0.1320,  0.0782],
        [ 0.2014,  0.0112, -0.2390, -0.2801,  0.1368,  0.2170,  0.2230,  0.4712,
          0.5739,  0.2955,  0.0103, -0.1626, -0.5595, -0.2013,  0.3525, -0.2189,
          0.4054, -0.2939, -0.0748,  0.1193],
        [ 0.0334, -0.0727, -0.0776,  0.0452,  0.1756, -0.1047,  0.2135,  0.5732,
          0.4895,  0.3364,  0.1317, -0.1718, -0.6371, -0.0717,  0.1173, -0.1712,
         -0.0803, -0.2407, -0.1719,  0.1489]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2871, 0.0000, 0.0000, 0.0446, 0.0000, 0.0000, 0.4203, 0.7009, 0.3356,
         0.3799, 0.0000, 0.0000, 0.0000, 0.0238, 0.2320, 0.0000, 0.3813, 0.0000,
         0.0000, 0.0782],
        [0.2014, 0.0112, 0.0000, 0.0000, 0.1368, 0.2170, 0.2230, 0.4712, 0.5739,
         0.2955, 0.0103, 0.0000, 0.0000, 0.0000, 0.35

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(in_features=20, out_features=10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0110,  0.0232,  0.0284,  ..., -0.0043,  0.0176,  0.0046],
        [-0.0262,  0.0010,  0.0312,  ...,  0.0332,  0.0230,  0.0178]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0355, -0.0200], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0419,  0.0260,  0.0200,  ...,  0.0351, -0.0377,  0.0340],
        [-0.0289,  0.0221,  0.0286,  ..., -0.0377,  0.0119,  0.0069]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: 